## Problem 1

In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import pandas as pd

# df1 = pd.read_csv('clickbait.txt', sep='\n')

# df2 = pd.read_csv('not-clickbait.txt', sep='\n')



with open('clickbait.txt', 'r', encoding='utf-8') as file:
    
    c=file.read()


with open('not-clickbait.txt', 'r', encoding='utf-8') as file:
    
    nc=file.read()

In [2]:
array_c=np.array(c.split('\n')[0:-1])
array_n_c=np.array(nc.split('\n')[0:-1])
p=np.full(len(array_c),1)
n=np.full(len(array_n_c),0)
sc=pd.Series(array_c)
snc=pd.Series(array_n_c)

In [3]:
dfc=pd.DataFrame(columns=["sente","targets"])
dfnc=pd.DataFrame(columns=["sente","targets"])
dfc.sente=sc
dfnc.sente=snc
dfnc.targets=n
dfc.targets=p

In [4]:
dfm=dfc.append(dfnc)

C:\Users\raghu\AppData\Local\Temp\ipykernel_18752\1968206897.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfm=dfc.append(dfnc)


In [5]:
dfm.value_counts(dfm.targets)

targets
0    1574
1     814
dtype: int64

In [6]:

shuffled_indices = np.random.permutation(dfm.index)
shuffled_df = dfm.iloc[shuffled_indices]

shuffled_df.reset_index(drop=True, inplace=True)

print(shuffled_df)

                                                  sente  targets
0     US telecoms giants call on FCC to block cities...        0
1     A guy walked into a restaurant kitchen and tur...        1
2     Experts Believe That A Stone Is Being Mistaken...        1
3     You won't believe what the new Baywatch has di...        1
4               Is 29 February a capitalist conspiracy?        1
...                                                 ...      ...
2383  The three biggest mistakes Hillary Clinton is ...        1
2384  Danny Green has an update on the Tim Duncan re...        1
2385  Applebee's says it figured out its biggest mis...        1
2386            U.S. running out of room to store crude        0
2387  Woman said safety was reason she didn't pull o...        0

[2388 rows x 2 columns]


In [7]:
from sklearn.model_selection import train_test_split
train_ratio = 0.72
validation_ratio = 0.08
test_ratio = 0.2


train_df, temp_df = train_test_split(shuffled_df, test_size=1 - train_ratio, random_state=42)
validation_df, test_df = train_test_split(temp_df, test_size=test_ratio / (test_ratio + validation_ratio), random_state=42)

In [8]:
print("train target rate",train_df.value_counts(train_df.targets)[0]/train_df.value_counts(train_df.targets).sum())
print("validation target rate",validation_df.value_counts(validation_df.targets)[0]/validation_df.value_counts(validation_df.targets).sum())
print("test target rate",test_df.value_counts(test_df.targets)[0]/test_df.value_counts(test_df.targets).sum())

train target rate 0.3106457242582897
validation target rate 0.387434554973822
test target rate 0.3179916317991632


In [9]:
train_df.head()

,sente,targets
207,THIS Powerful Gesture Will Stop Your Kid From ...,1
2082,Did #TheBachelor Ben Higgins get an offer to c...,1
1146,The three biggest mistakes Hillary Clinton is ...,1
729,3 Things You Should Never Give To Charity,1
162,Here are the 10 countries where homosexuality ...,1


## Problem 3

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 2))),
    ('classifier', MultinomialNB())
])


model=pipeline.fit(list(train_df.sente), list(train_df.targets))

In [11]:
val_ans=model.predict(list(validation_df.sente))
test_ans=model.predict(list(test_df.sente))

In [12]:
from sklearn.metrics import f1_score ,precision_score, recall_score
f1_val=f1_score(np.array(list(validation_df.targets)), val_ans, average='binary')


In [13]:
precision_val = precision_score(np.array(list(validation_df.targets)), val_ans)

recall_vall = recall_score(np.array(list(validation_df.targets)), val_ans)

In [14]:
precision_test = precision_score(np.array(list(test_df.targets)), test_ans)

recall_test = recall_score(np.array(list(test_df.targets)), test_ans)
f1_test=f1_score(np.array(list(test_df.targets)), test_ans, average='binary')

In [15]:
print("validation precision",precision_val)
print("validation recall",recall_vall)
print("validation f1",f1_val)

validation precision 0.8863636363636364
validation recall 1.0
validation f1 0.9397590361445783


In [16]:
print("test precision",precision_test)
print("test recall",recall_test)
print("test f1",f1_test)

test precision 0.9002770083102493
test recall 0.9969325153374233
test f1 0.9461426491994177


## Problem 4

In [17]:
from sklearn.model_selection import ParameterGrid
p_grid = {
    'max_df': [0.1,0.3,0.5, 0.7, 0.9],  
    'alpha': [0.1,0.2,0.3,0.5,0.6,0.75, 0.9],  
    'ngram': [ (1,3),(1,1),(1,2),(2,2),(3,3)]  
}


In [18]:
grid = ParameterGrid(p_grid)

In [19]:
len(list(train_df.sente))

1719

In [20]:
from sklearn.metrics import accuracy_score
bp=[]
pr=[]
rl=[]
f1=[]
ba=[]

for i in grid:
    pipeline = Pipeline([
        ('vectorizer',CountVectorizer(max_df=i['max_df'],ngram_range=i['ngram'])),
        ('classifier', MultinomialNB(alpha=i['alpha']))
    ])
    bp.append(i)   
    model=pipeline.fit(list(train_df.sente), list(train_df.targets))
    
    val_ans=model.predict(list(validation_df.sente))
    f1_val=accuracy_score(np.array(list(validation_df.targets)), val_ans)
    ba.append(f1_val)
    precision_val = precision_score(np.array(list(validation_df.targets)), val_ans)
    
    recall_vall = recall_score(np.array(list(validation_df.targets)), val_ans)
    pr.append(precision_val)
    rl.append(recall_vall)
    
    

In [21]:
maxxx=np.array(ba).argmax(axis=0)
maxx=5

In [22]:
print("the best paramenters",bp[maxxx])

the best paramenters {'alpha': 0.1, 'max_df': 0.1, 'ngram': (1, 3)}


In [23]:
print("precision",pr[maxxx],"recall",rl[maxxx],"accuracy",ba[maxxx])

precision 0.9285714285714286 recall 1.0 accuracy 0.9528795811518325


## Problem 5

In [24]:
bp[maxx]

{'alpha': 0.1, 'max_df': 0.3, 'ngram': (1, 3)}

In [25]:
pipeline = Pipeline([
        ('vectorizer',CountVectorizer(max_df=bp[maxx]['max_df'],ngram_range=bp[maxx]['ngram'])),
        ('classifier', MultinomialNB(alpha=bp[maxx]['alpha']))
    ])

model=pipeline.fit(list(train_df.sente), list(train_df.targets))


In [26]:
test_ans=model.predict(list(test_df.sente))
precision_test = precision_score(np.array(list(test_df.targets)), test_ans)

recall_test = recall_score(np.array(list(test_df.targets)), test_ans)
f1_test=f1_score(np.array(list(test_df.targets)), test_ans, average='binary')

In [27]:
print("precision",precision_test,"recall",recall_test,"f1 score",f1_test)

precision 0.948170731707317 recall 0.9539877300613497 f1 score 0.9510703363914372


## Problem 6

In [28]:

mlp=model['classifier'].feature_log_prob_[1]-model['classifier'].feature_log_prob_[0]

In [29]:
len(mlp)

26862

In [30]:
mlp

array([-1.86473795,  1.61861153,  1.61861153, ...,  2.26523869,
        2.26523869,  2.26523869])

In [31]:
mlw=model['vectorizer'].get_feature_names_out()

In [32]:
mlw

array(['000', '000 000', '000 000 to', ..., 'zuckerberg',
       'zuckerberg put', 'zuckerberg put tape'], dtype=object)

In [33]:
len(mlw)

26862

In [34]:
df_6=pd.DataFrame(columns=["log_probablities","words"])

In [35]:
df_6.log_probablities=mlp
df_6.words=mlw

In [36]:
df_6

,log_probablities,words
0,-1.864738,000
1,1.618612,000 000
2,1.618612,000 000 to
3,-3.177179,000 boston
4,-3.177179,000 boston public
...,...,...
26857,2.265239,zooey deschanel
26858,2.265239,zooey deschanel has
26859,2.265239,zuckerberg
26860,2.265239,zuckerberg put


In [37]:
df_6_n=df_6.sort_values('log_probablities',ascending=False)

In [38]:
df_6_n[:10]

,log_probablities,words
26710,5.815130,you won believe
26216,5.815130,won believe
26709,5.815130,you won
9747,5.602532,here
26616,5.532451,you ll
26618,5.514136,you ll never
12917,5.514136,ll never
2674,5.437322,believe what
8988,5.396584,guess
22642,5.286824,this is


## Problem 7

In [39]:
w_5=list(df_6_n[:5].words)

In [40]:
w_5

['you won believe', 'won believe', 'you won', 'here', 'you ll']

In [41]:
import re

In [42]:
ws="hi wood_chuck the wonder"
print(re.search(r"\bwood_chuck\b",ws,re.IGNORECASE))

<re.Match object; span=(3, 13), match='wood_chuck'>


In [43]:
test_df_reg=pd.DataFrame(columns=['sente','targets','pred'])

In [44]:
test_df_reg.sente=test_df.sente.copy()
test_df_reg.targets=test_df.targets.copy()

In [45]:
check_word="won believe"

In [46]:
predd=[]
for i in range(0,len(test_df_reg)):
    s=test_df_reg[i:i+1]
    str_chk=list(s.sente)[0]
    test=re.search(r"\byou won\b",str_chk,re.IGNORECASE)
    if(test==None):
        predd.append(0)
    else:
        predd.append(1)
        
    

In [47]:
test_df_reg.targets.value_counts()

1    326
0    152
Name: targets, dtype: int64

In [48]:
precision_test_7 = precision_score(np.array(list(test_df.targets)),predd )

recall_test_7 = recall_score(np.array(list(test_df.targets)), predd)
f1_test_7=f1_score(np.array(list(test_df.targets)), predd, average='binary')

In [49]:
print("precision",precision_test_7,"recall",recall_test_7,"f1 score",f1_test_7)

precision 1.0 recall 0.05214723926380368 f1 score 0.09912536443148688
